In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
class system():
    
    def __init__(self, name, mass, pos=np.array([[0,0,0]])):
        
        self.bodies = pd.DataFrame(col=['Name','Position','Mass','Density'])
        self.positions = pos.T #numpy array of shape dimension 2 or 3, number of objects
        self.central_pos = pos
        self.central_name = name
        self.central_mass = mass      
        
        
        pass
    
    
    def add_body(self, name, mass, pos, velocity):
        
        new_row = {'Name': name, 'Position': pos, 'Mass': mass, 'Velocity': velocity}
        
        self.bodies.append(new_row)
        
        pass
    
    def return_positions()
    
        return self.positions
    
    def update_kinematics(self, ):
        
        pass
    
    def delete_body(self, body):
        
        pass
    
    def move(self, time):
        
        pass
    
    def run_sim(self, n_iteration):
        
        pass
    

In [20]:
test_sys = system('Sun', 1)

print(test_sys.positions)

[[0]
 [0]
 [0]]


In [19]:
class celestial_body():
    
    def __init__(name, pos, mass, velocity):
        
        self.name = name #string
        self.position = np.array(pos) #list
        self.mass = mass #integer
        self.velocity = np.array(velocity) #list of component velocities
        
    
    